# Hand gesture recognition

---


## 1. Import packages


In [1]:
import threading
%pip install -r requirements.txt

Note: you may need to restart the kernel to use updated packages.


In [2]:
import cv2
import mediapipe as mp

## 2. Create utils functions


### OpenCV


### MediaPipe


In [3]:
MIN_DETECTION_CONFIDENCE = 0.4
MIN_PRESENCE_CONFIDENCE = 0.4
NUM_HANDS = 1
MIN_TRACKING_CONFIDENCE = 0.3

In [4]:
mp_holistic = mp.solutions.holistic
mp_drawing = mp.solutions.drawing_utils

### Model predictions


In [5]:
labels = [
    "closed",
    "dislike",
    "like",
    "palm",
    "point_up",
    "rock",
    "victory",
    "victory_inverted",
]

## 3. Preprocess the data


In [6]:
def frame_preprocessing(frame, resolution=None, flip=False):
    if resolution is not None:
        frame = cv2.resize(frame, resolution)

    if flip:
        frame = cv2.flip(frame, 1)

    return frame

## Execute program


### Websocket server

### Tweak settings


#### Camera Settings


In [7]:
DEBUG = False
FRAMERATE = None
RESOLUTION = (833, 480)  # (1280, 720)  # None  # (833, 480)  # (1280, 720)
FLIP_CAMERA = False

#### Mediapipe hand bones


In [8]:
BOX_MARGIN = 24

MIN_DETECTION_CONFIDENCE = 0.4
MIN_PRESENCE_CONFIDENCE = 0.4
NUM_HANDS = 1
MIN_TRACKING_CONFIDENCE = 0.3

#### Model settings


In [9]:
MODEL_NAME = "4_feb_w_additional_datasets"

MODEL_PATH = f"./models/{MODEL_NAME}/{MODEL_NAME}.hdf5"
MIN_GESTURE_CONFIDENCE = 0.7
MP_MODEL_COMPLEXITY = 0

In [10]:
HAND_CONTROL = "right"
SWIPE_THRESHOLD = 0.2

In [11]:
from helpers.websockets.send_gesture import send_gesture
import asyncio
from helpers.gesture_handler.gesture_handler import GestureHandler
from helpers.computations import compute_hand_coordinates
from tensorflow.keras.models import load_model

from helpers.predictions import get_gesture
from helpers.mediapipe import get_landmarks, draw_landmarks, draw_box
from helpers.camera import read_frame, show_frame, get_close_event, close_camera, draw_hand_pointer, flip_frame

gesture_handler = GestureHandler(frame_resolution=RESOLUTION, swipe_threshold=SWIPE_THRESHOLD)

capture = cv2.VideoCapture(1)

model = load_model(MODEL_PATH, compile=False)

hand = [0, 1] if HAND_CONTROL == "right" else [1, 0]

with mp_holistic.Holistic(
        min_detection_confidence=MIN_DETECTION_CONFIDENCE,
        min_tracking_confidence=MIN_TRACKING_CONFIDENCE,
        model_complexity=MP_MODEL_COMPLEXITY,
) as holistic:
    while True:
        frame = read_frame(capture, FRAMERATE)
        frame = frame_preprocessing(frame, RESOLUTION, FLIP_CAMERA)

        landmarks = get_landmarks(frame, holistic)

        hand_landmark = landmarks[HAND_CONTROL]

        current_swipe = None

        if hand_landmark:
            if DEBUG: frame = draw_landmarks(frame, hand_landmark, mp_holistic, mp_drawing)

            # Get hand coordinates (not normalized)
            hand_coordinates = compute_hand_coordinates(frame, hand_landmark)

            # Draw hand pointer
            frame = draw_hand_pointer(frame, hand_coordinates)

            # Get current gesture
            gesture, accuracy = get_gesture(
                model, labels, MIN_GESTURE_CONFIDENCE, hand, hand_landmark
            )

            # Draw box around hand
            frame = draw_box(frame, gesture, accuracy, hand, hand_landmark)

            # Handle gesture logic
            current_swipe = gesture_handler.listen(frame, hand_coordinates, gesture)

            print(current_swipe)

            payload = {
                "type": "gesture",
                "gesture": gesture,
                "accuracy": accuracy,
                "hand": hand,
                "swipe": current_swipe,
            }

            loop = asyncio.get_event_loop()
            loop.create_task(send_gesture(gesture))
    
        frame = flip_frame(frame)

        # Display swipe on frame
        gesture_handler.draw_swipe(frame, current_swipe)
        show_frame(frame, "hand gesture recognition")

        if get_close_event():
            break

close_camera(capture)

2024-03-25 17:02:26.815 python[2336:17561] WARNING: AVCaptureDeviceTypeExternal is deprecated for Continuity Cameras. Please use AVCaptureDeviceTypeContinuityCamera and add NSCameraUseContinuityCameraDeviceType to your Info.plist.
I0000 00:00:1711382548.196067       1 gl_context.cc:344] GL version: 2.1 (2.1 Metal - 88), renderer: Apple M1 Pro
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


None
None
None
None
None
None
None
None
None
None
None
Locking...
hover_none
hover_none
hover_none
hover_down-right
hover_down-right
hover_down-right
hover_down-right
hover_down-right
Unlocking...
down-right
None
None
None
None
None
None
None
Locking...
hover_none
hover_right
hover_down-right
hover_down-right
hover_down-right
Unlocking...
down-right
None
None
None
Locking...
hover_none
hover_none
hover_down-right
hover_down-right
Unlocking...
down-right
None
None
None
None
Locking...
hover_none
hover_none
hover_down
hover_down
hover_down-right
Unlocking...
down-right
None
None
None
Locking...
hover_none
hover_none
hover_down-left
hover_down-left


KeyboardInterrupt: 